In [0]:
select * from gizmobox.bronze.v_orders

In [0]:
create or replace temporary view tv_orders as
select 
  value,
  regexp_replace(value, '"order_date": ([0-9]{4}-[0-9]{2}-[0-9]{2})', '"order_date": "\$1"') as fixed_value
from gizmobox.bronze.v_orders

In [0]:
select 
  schema_of_json(fixed_value)
from tv_orders
limit 1;

In [0]:
select from_json(fixed_value, 
                'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as json_value,
        fixed_value
from tv_orders

In [0]:
create table if not exists gizmobox2.silver2.orders_json
as 
select from_json(fixed_value, 
                'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as json_value
from tv_orders

In [0]:
select * from gizmobox2.silver2.orders_json